In [8]:
import sys
sys.path.append('..')
from utils.forestriesz import ForestRieszATE
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [3]:
# simulate data
def sim_data(n, gamma=0.5, counter_A=None):
    # error
    UY = np.random.normal(loc=0, scale=np.sqrt(0.5), size=n)

    # baseline covariates
    W1 = np.round(np.random.uniform(-1, 1, size=n), 2)
    W2 = np.round(np.random.uniform(-1, 1, size=n), 2)
    W3 = np.round(np.random.uniform(-1, 1, size=n), 2)
    W4 = np.round(np.random.uniform(-1, 1, size=n), 2)

    # treatment
    if counter_A is None:
        logits = gamma * (W1 + W2 + W3 + W4 +
                          np.sin(4 * W1) + np.sin(4 * W2) +
                          np.sin(4 * W3) + np.sin(4 * W4))
        A = np.random.binomial(n=1, p=1 / (1 + np.exp(-logits)))
    else:
        A = np.full(n, counter_A)

    # outcome
    Y = (np.cos(4 * W2) + np.sin(4 * W1) + np.sin(4 * W2) +
         np.sin(4 * W3) + np.sin(4 * W4) +
         A * (1 + W1 + np.abs(W2) + np.cos(4 * W3) + W4) + UY)

    data = pd.DataFrame({
        'W1': W1,
        'W2': W2,
        'W3': W3,
        'W4': W4,
        'A': A,
        'Y': Y
    })

    return data

def get_truth():
    data_A1 = sim_data(int(1e7), counter_A=1)
    data_A0 = sim_data(int(1e7), counter_A=0)
    return np.mean(data_A1['Y'] - data_A0['Y'])

In [ ]:
n = 1000
n_sim = 10
results = []
for i in range(n_sim):
    data = sim_data(n)
    model = ForestRiesz(data, 'Y', 'A', ['W1', 'W2', 'W3', 'W4'])
    model.fit()
    results.append(model.estimate())

In [ ]:
res_df = []
n_sim = 10
for i in range(n_sim):
    print(f"Simulation {i+1}/{n_sim}")
    n = 5000
    data = sim_data(n)
    # Extract features and create X
    X = data[['A', 'W1', 'W2', 'W3', 'W4']].values 
    y = data['Y'].values.reshape(-1, 1)
    y_scaler = StandardScaler(with_mean=True).fit(y)
    y = y_scaler.transform(y)
    est = ForestRieszATE(criterion='mse', n_estimators=1000, min_samples_leaf=2,
                        min_var_fraction_leaf=0.001, min_var_leaf_on_val=True,
                        min_impurity_decrease = 0.01, max_samples=.8, max_depth=None,
                        warm_start=False, inference=False, subforest_size=1,
                        honest=True, verbose=0, n_jobs=-1, random_state=123)
    est.fit(X[:, 1:], X[:, [0]], y.reshape(-1, 1))
    res = est.predict_ate(X, y, method='tmle')
    psi = res[0]
    lower = res[1]
    upper = res[2]
    res_df.append([psi, lower, upper])

Simulation 1/10
Simulation 2/10
Simulation 3/10
Simulation 4/10
Simulation 5/10
Simulation 6/10
Simulation 7/10
Simulation 8/10
Simulation 9/10
Simulation 10/10


In [24]:
res_df = pd.DataFrame(res_df, columns=['psi', 'lower', 'upper'])

In [21]:
res_df

,psi,lower,upper
0,0.706414,0.634096,0.778732
1,0.706769,0.631444,0.782095
2,0.694129,0.624551,0.763708
3,0.740220,0.668925,0.811515
4,0.602782,0.529993,0.675571
5,0.737105,0.667621,0.806588
6,0.729330,0.657585,0.801075
7,0.640524,0.572603,0.708444
8,0.659168,0.587916,0.730419
9,0.681180,0.608795,0.753566


In [22]:
get_truth()

1.3112417193876098

In [ ]:
    
# save results to a CSV file
pd.DataFrame(results).to_csv('results.csv', index=False)